In [1]:
from torch.utils.data import Dataset, DataLoader# For custom data-sets
import torchvision.transforms as transforms
import numpy as np
from PIL import Image
import torch
import pandas as pd
from collections import namedtuple, defaultdict
import random
from tqdm import tqdm 
n_class    = 27

# a label and all meta information
Label = namedtuple( 'Label' , [
    'name'        , 
    'level3Id'    , 
    'color'       , 
    ] )

labels = [
    #       name                     level3Id  color
    Label(  'road'                 ,    0  , (128, 64,128)  ),
    Label(  'drivable fallback'    ,    1  , ( 81,  0, 81)  ),
    Label(  'sidewalk'             ,    2  , (244, 35,232)  ),
    Label(  'non-drivable fallback',    3  , (152,251,152)  ),
    Label(  'person/animal'        ,    4  , (220, 20, 60)  ),
    Label(  'rider'                ,    5  , (255,  0,  0)  ),
    Label(  'motorcycle'           ,    6  , (  0,  0,230)  ),
    Label(  'bicycle'              ,   7  , (119, 11, 32)  ),
    Label(  'autorickshaw'         ,   8  , (255, 204, 54) ),
    Label(  'car'                  ,   9  , (  0,  0,142)  ),
    Label(  'truck'                ,  10 ,  (  0,  0, 70)  ),
    Label(  'bus'                  ,  11 ,  (  0, 60,100)  ),
    Label(  'vehicle fallback'     ,  12 ,  (136, 143, 153)),  
    Label(  'curb'                 ,   13 ,  (220, 190, 40)),
    Label(  'wall'                 ,  14 ,  (102,102,156)  ),
    Label(  'fence'                ,  15 ,  (190,153,153)  ),
    Label(  'guard rail'           ,  16 ,  (180,165,180)  ),
    Label(  'billboard'            ,   17 ,  (174, 64, 67) ),
    Label(  'traffic sign'         ,  18 ,  (220,220,  0)  ),
    Label(  'traffic light'        ,  19 ,  (250,170, 30)  ),
    Label(  'pole'                 ,  20 ,  (153,153,153)  ),
    Label(  'obs-str-bar-fallback' , 21 ,  (169, 187, 214) ),  
    Label(  'building'             ,  22 ,  ( 70, 70, 70)  ),
    Label(  'bridge/tunnel'        ,  23 ,  (150,100,100)  ),
    Label(  'vegetation'           ,  24 ,  (107,142, 35)  ),
    Label(  'sky'                  ,  25 ,  ( 70,130,180)  ),
    Label(  'unlabeled'            ,  26 ,  (  0,  0,  0)  ),
]

In [2]:
class Resize(object):
    def __init__(self, output_size): # the output size should be a tuple of exactly (1080, 1920)
        self.output_size = output_size
        self.imageResize = transforms.Resize(self.output_size, Image.BILINEAR)
        self.labelResize = transforms.Resize(self.output_size, Image.NEAREST)
    def __call__(self, sample):
        image, label = sample
        img = self.imageResize(image)
        lab = self.labelResize(label)
        return img, lab
class MirrorFlip(object):
    def __init__(self, probability = 0.5):
        self.hFlip = transforms.functional.hflip
        self.prob = probability
    def __call__(self, sample):
        image, label = sample
        if random.random() > self.prob:
            img = self.hFlip(image)
            lab = self.hFlip(label)
        else:
            img = image
            lab = label
        return img, lab
class Rotate(object):
    def __init__(self, degree = 10):
        self.degree = degree
        self.rotation = transforms.functional.rotate
    def __call__(self, sample):
        image, label = sample
        imageDegree = self.degree * random.random() - self.degree/2
        img = self.rotation(image, angle=imageDegree, resample=Image.BILINEAR)
        lab = self.rotation(label, angle=imageDegree, resample=Image.NEAREST)
        return img, lab

class RondomCrop(object):
    def __init__(self, shape):
        self.shape = shape
    def __call__(self, sample):
        image, label = sample
        a, b, c, d = transforms.RandomCrop.get_params(image, output_size=self.shape)
        img = transforms.functional.crop(image, a, b, c, d)
        lab = transforms.functional.crop(label, a, b, c, d)
        return img,lab
    
class Blur(object):
    def __init__(self, kernel_size = 3, sigma=(0.1, 1)):
        self.kernel_size = kernel_size
        self.sigma = sigma
        self.blurr = transforms.GaussianBlur(self.kernel_size, self.sigma)
    def __call__(self, sample):
        image, lab = sample
        self.blurr(image)
        return image, lab
    
class ToTensor(object):
    def __init__(self):
        self.transform = transforms.ToTensor()
    def __call__(self, sample):
        image, label = sample
        image = self.transform(image)
        label = torch.from_numpy(np.asarray(label))
        return image, label
class CenterCrop(object):
    def __init__(self, size):
        self.size = size
        self.transform = transforms.CenterCrop(size)
    def __call__(self, sample):
        image, label = sample
        img = self.transform(image)
        label = self.transform(label)
        return img, label
class Normalize(object):
    def __init__(self):
        self.transform = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    def __call__(self, sample):
        image, label = sample
        image = self.transform(image)
        return image, label

In [3]:
class IddDataset(Dataset):
    def __init__(self, csv_file, n_class=n_class, transforms_=None):
        self.data      = pd.read_csv(csv_file)
        self.n_class   = n_class
        self.mode = csv_file
        
        # Add any transformations here
        
        # The following transformation normalizes each channel using the mean and std provided
        self.transforms = transforms.Compose([
                                            Resize((512, 960)),
                                            #RondomCrop((1024, 1320)),
                                            #Blur(),
                                            #CenterCrop((600, 900)),
                                            #Rotate(),
                                            #MirrorFlip(),
                                            ToTensor(),
                                            #Normalize(),
                                            ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        
        img_name = self.data.iloc[idx, 0]
        img = Image.open(img_name).convert('RGB')
        label_name = self.data.iloc[idx, 1]
        label = Image.open(label_name)

        #print(trans(label = np.asarray(label)))
        
        
        img, label = self.transforms((img, label)) # Normalization
        #label = torch.from_numpy(label.copy()).long() # convert to tensor

        # create one-hot encoding
        
        label = torch.squeeze(label)
        
        #h, w = label.shape
        #target = torch.zeros(self.n_class, h, w)
        #for c in range(self.n_class):
        #    target[c][c==label] = 1
        return img, label
    def classWeightCalculate(self):
        self.classWeight = defaultdict(int)
        for idx in tqdm(range(len(self.data)), desc="Loading..."):
            label_name = self.data.iloc[idx, 1]
            label = np.asarray(Image.open(label_name))
            values, counts = np.unique(label, return_counts=True)
            for i in range(len(values)):
                self.classWeight[values[i]] += counts[i]
        return self.classWeight
    def weightedLossCalculate(self):
        if self.weightLoss:
            return self.weightLoss
        if not self.classWeight:
            self.classWeightCalculate()
        total = 0
        maxKey = 0
        for key, value in self.classWeight.items():
            maxKey = max(maxKey, key)
            total += value
        self.weightLoss = []
        for i in range(maxKey):
            self.weightLoss.append(total/self.classWeight[i])
        return torch.tensor(np.array(self.weightLoss))

In [4]:
Idd = IddDataset('train.csv')


Loading...: 100%|██████████| 4018/4018 [02:43<00:00, 24.64it/s]


defaultdict(int,
            {0: 2197668581,
             4: 58799437,
             5: 59027721,
             6: 94290243,
             8: 101519467,
             9: 163599994,
             11: 47400301,
             12: 30998126,
             17: 104772033,
             20: 94801607,
             21: 341772622,
             22: 557014134,
             24: 1464952370,
             25: 1823217300,
             26: 2775322,
             1: 519519185,
             3: 126353932,
             13: 133754816,
             14: 135933370,
             15: 41339482,
             18: 8062798,
             2: 28929892,
             10: 105052002,
             16: 15989829,
             19: 450973,
             23: 71159311,
             7: 2569952})

In [13]:

Idd.weightedLossCalculate()

[3.791165270337912,
 16.03737656002059,
 287.9970931104755,
 65.93957677549757,
 141.69735672809247,
 141.14935591025105,
 88.36253396865251,
 3241.976815131178,
 82.07021811885596,
 50.927415070687594,
 79.31048091782202,
 175.77366861024785,
 268.7815644081194,
 62.29102659002574,
 61.29271127464875,
 201.54400580055648,
 521.0640338930453,
 79.52241224526014,
 1033.354029209215,
 18474.99695103698,
 87.88590260922476,
 24.37797606854536,
 14.9578337270702,
 117.08551815517157,
 5.687369071255197,
 4.569792531038401]

In [5]:
from PIL import Image
import matplotlib.pyplot as plt
trans = transforms.ToPILImage()
#plt.imshow(trans((Idd[0][0])))
print(Idd[0][1])


tensor([[22, 22, 22,  ..., 20, 20, 22],
        [22, 22, 22,  ..., 20, 20, 22],
        [22, 22, 22,  ..., 20, 20, 22],
        ...,
        [ 8,  8,  8,  ...,  4,  4,  4],
        [ 8,  8,  8,  ...,  4,  4,  4],
        [ 8,  8,  8,  ...,  4,  4,  4]], dtype=torch.uint8)


In [6]:
dl = DataLoader(Idd, batch_size=16, num_workers=2)

In [7]:
for i, (img, label) in enumerate(dl):
    print(i)
    #trans = transforms.ToPILImage()
    #plt.imshow(trans(img[0]))
    #plt.show()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181


KeyboardInterrupt: 